In [ ]:
import pandas as pd

# Load the dataset
try:
    df = pd.read_csv('final_food_delivery_dataset.csv')
except:
    # Fallback to regeneration
    orders_df = pd.read_csv('orders.csv')
    users_df = pd.read_json('users.json')
    import re
    restaurant_data = []
    with open('restaurants.sql', 'r') as f:
        for line in f:
            if line.strip().startswith("INSERT INTO restaurants VALUES"):
                match = re.search(r"\((.*)\);", line)
                if match:
                    restaurant_data.append([p.strip().strip("'") for p in match.group(1).split(',')])
    restaurants_df = pd.DataFrame(restaurant_data, columns=['restaurant_id', 'restaurant_name', 'cuisine', 'rating'])
    restaurants_df['restaurant_id'] = pd.to_numeric(restaurants_df['restaurant_id'])
    merged = pd.merge(orders_df, users_df, on='user_id', how='left')
    df = pd.merge(merged, restaurants_df, on='restaurant_id', how='left', suffixes=('_order', '_details'))

# Group by membership and cuisine
revenue_by_combo = df.groupby(['membership', 'cuisine'])['total_amount'].sum().reset_index()

# Define the options to check
options = [
    ('Gold', 'Indian'),
    ('Gold', 'Italian'),
    ('Regular', 'Indian'),
    ('Regular', 'Chinese')
]

# Filter and display relevant rows
print("Revenue for each option:")
for mem, cuis in options:
    val = revenue_by_combo[(revenue_by_combo['membership'] == mem) & (revenue_by_combo['cuisine'] == cuis)]
    if not val.empty:
        print(f"{mem} + {cuis}: {val['total_amount'].values[0]}")
    else:
        print(f"{mem} + {cuis}: Not found")

# Just to be sure, print all sorted
print("\nAll combinations sorted:")
print(revenue_by_combo.sort_values('total_amount', ascending=False))